In [1]:

from typing import Any, Callable, List, Optional, Tuple, Union, Dict
from torch import Tensor
from torchvision.ops import MultiScaleRoIAlign
from torchvision.models.detection.mask_rcnn import MaskRCNNHeads, MaskRCNNPredictor

In [ ]:
from torchvision.models.detection.roi_heads import RoIHeads
from torch import nn

class RoIHeadsWithNocs(nn.Module):
    def __init__(self, in_channels, num_bins=32, other_heads:RoIHeads=None):
        super().__init__()
        self.heads = other_heads

        layers = (256, 256, 256, 256, 256)

        self.nocs_heads = {}
        for k in ['x', 'y', 'z']:
            self.nocs_heads[k] = nn.ModuleDict({
                'roi_align': MultiScaleRoIAlign(
                                featmap_names=["0", "1", "2", "3"], 
                                output_size=14, 
                                sampling_ratio=2),
                'head': MaskRCNNHeads(
                                in_channels, 
                                layers[:-1], 
                                dilation=1),
                'pred': MaskRCNNPredictor(
                                layers[-2], 
                                layers[-1], 
                                num_bins)
            })

    def forward(
            self,
            features     : Dict[str, Tensor],
            proposals    : List[Tensor],
            image_shapes : List[Tuple[int, int]],
            targets      : Optional[List[Dict[str, Tensor]]] = None ,
            ):
        
        results, lables = self.heads(features, proposals, image_shapes, targets)
        for _, head in self.nocs_heads.items():
            x = head['roi_align'](features, proposals, image_shapes)
            x = head['head'](x)
            x = head['pred'](x)
        return results, lables

In [3]:
from torchvision.models.detection.mask_rcnn import MaskRCNN


class NOCS(MaskRCNN):
    def __init__(
        self,
        backbone,
        num_classes=None,
        # transform parameters
        min_size=800,
        max_size=1333,
        image_mean=None,
        image_std=None,
        # RPN parameters
        rpn_anchor_generator=None,
        rpn_head=None,
        rpn_pre_nms_top_n_train=2000,
        rpn_pre_nms_top_n_test=1000,
        rpn_post_nms_top_n_train=2000,
        rpn_post_nms_top_n_test=1000,
        rpn_nms_thresh=0.7,
        rpn_fg_iou_thresh=0.7,
        rpn_bg_iou_thresh=0.3,
        rpn_batch_size_per_image=256,
        rpn_positive_fraction=0.5,
        rpn_score_thresh=0.0,
        # Box parameters
        box_roi_pool=None,
        box_head=None,
        box_predictor=None,
        box_score_thresh=0.05,
        box_nms_thresh=0.5,
        box_detections_per_img=100,
        box_fg_iou_thresh=0.5,
        box_bg_iou_thresh=0.5,
        box_batch_size_per_image=512,
        box_positive_fraction=0.25,
        bbox_reg_weights=None,
        # Mask parameters
        mask_roi_pool=None,
        mask_head=None,
        mask_predictor=None,
        **kwargs,
    ):
        super().__init__(
            backbone,
            num_classes,
            # transform parameters
            min_size,
            max_size,
            image_mean,
            image_std,
            # RPN-specific parameters
            rpn_anchor_generator,
            rpn_head,
            rpn_pre_nms_top_n_train,
            rpn_pre_nms_top_n_test,
            rpn_post_nms_top_n_train,
            rpn_post_nms_top_n_test,
            rpn_nms_thresh,
            rpn_fg_iou_thresh,
            rpn_bg_iou_thresh,
            rpn_batch_size_per_image,
            rpn_positive_fraction,
            rpn_score_thresh,
            # Box parameters
            box_roi_pool,
            box_head,
            box_predictor,
            box_score_thresh,
            box_nms_thresh,
            box_detections_per_img,
            box_fg_iou_thresh,
            box_bg_iou_thresh,
            box_batch_size_per_image,
            box_positive_fraction,
            bbox_reg_weights,
            # Mask parameters
            mask_roi_pool,
            mask_head,
            mask_predictor,
            **kwargs,
        )
        self.roi_heads = RoIHeadsWithNocs(self.roi_heads)

In [4]:
import torch
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn, MaskRCNN_ResNet50_FPN_Weights

from pathlib import Path
import os

from torchvision import transforms


if 'image' not in vars():
    # DATA_FOLDER = Path(__file__).resolve().parent / 'data'
    DATA_FOLDER = Path('/home/bahaa/Code/torch_nocs/data')
    dataset = torchvision.datasets.CocoDetection(
                                DATA_FOLDER/'train' 
                                ,DATA_FOLDER/'annotations/instances_train2017.json'
                                ,transform=transforms.PILToTensor())
    image, targets = dataset[0][0], dataset[0][1] 

loading annotations into memory...
Done (t=19.39s)
creating index...
index created!


In [5]:
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models.detection.backbone_utils import _resnet_fpn_extractor, _validate_trainable_layers, resnet_fpn_backbone


backbone = resnet_fpn_backbone('resnet50', ResNet50_Weights.DEFAULT)
m = NOCS(backbone, 91)


m.eval()
x = [torch.rand(3, 30, 40), torch.rand(3, 50, 40)]
# x = [image.float() / 255.0]
predictions = m(x)

/home/bahaa/anaconda3/envs/nocs/lib/python3.8/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'backbone_name' and 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


: 

: 